# Data Science – Skills Assessment 
### Rohit Garg

---
# 1. import libraries

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
plt.style.use('seaborn-white')
plt.rcParams['figure.figsize'] = 15,5
%matplotlib inline

---
# 2. import dataset
* The raw file is imported 
* The count of rows is 100,000 and count of unique asset id is 82,442. 

In [3]:
df = pd.read_csv('DellGSPE_DSrole_dataset.csv')
print('Count of complaints: ',df.shape[0])
print('Count of unique assets: ',df['asst_id'].nunique())

Count of complaints:  100000
Count of unique assets:  82442


---
# 3. asset information
* Missing product type - Against few asset ids it was observed that the product type is missing. There are 82,423 unique asset ids where the product type is present 
* Missing region and country - There are 82,439 unique asset ids where the region is present. There are 75,910 unique asset ids where the country is present. 

In [4]:
dfa = df[['asst_id','product_type']]
dfa = dfa.dropna(subset = ['product_type'])
dfa['cnt'] = 0
dfa = dfa.groupby(['asst_id','product_type']).agg({'cnt':'sum'}).reset_index()
dfa.shape

(82423, 3)

In [5]:
dfb = df[['asst_id','mnfture_wk', 'contract_st', 'contract_end']]
dfb['cnt'] = 1
dfb = dfb.groupby(['asst_id','mnfture_wk', 'contract_st', 'contract_end']).agg({'cnt':'sum'}).reset_index()
dfb['repeat compaint'] = np.where(dfb['cnt']>1, 'multiple', 'single')
dfb.shape

(82442, 6)

In [6]:
dfc = df[['asst_id','region']]
dfc = dfc.dropna(subset = ['region'])
dfc['cnt'] = 0
dfc = dfc.groupby(['asst_id','region']).agg({'cnt':'sum'}).reset_index()
dfc.shape

(82439, 3)

In [7]:
dfd = df[['asst_id','country']]
dfd = dfd.dropna(subset = ['country'])
dfd['cnt'] = 0
dfd = dfd.groupby(['asst_id','country']).agg({'cnt':'sum'}).reset_index()
dfd.shape

(75910, 3)

In [8]:
dfe = pd.merge(dfa, dfb, how='outer', on='asst_id')
dfe = pd.merge(dfe, dfc, how='outer', on='asst_id')
dfe = pd.merge(dfe, dfd, how='outer', on='asst_id')
dfe = dfe.drop(['cnt_x', 'cnt_y'], axis=1)
dfe.shape

(82442, 8)

In [9]:
dfe['product_type'] = dfe['product_type'].fillna('Missing')
dfe['country'] = dfe['country'].fillna('Missing')
dfe['region'] = dfe['region'].fillna('Missing')
dfe.shape

(82442, 8)

---
# 4. master table - assets
* Relevant columns are selected for the analysis

In [10]:
df = df[['asst_id','contact_wk','contact_type','issue_type','repair_type','contact_manager_flg','diagnostics']]
df = pd.merge(df, dfe, how='left', on='asst_id')
df['outside contract'] = np.where((df['contact_wk']<df['contract_st'])|(df['contact_wk']>df['contract_end']), 'outside', 'inside')
df.shape

(100000, 15)

---
# 5. analysis 1
* Product type, Region and Country 
* The data is rolled up at Product type and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Region and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Country and count of asset id is taken (the sum of count of asset id is 100,000).

In [11]:
gb = df.groupby('product_type').agg({'asst_id':'count'}).reset_index()
gb

,product_type,asst_id
0,Desktops,23601
1,Laptops,73053
2,Missing,19
3,Other Electronics,3327


In [12]:
gb = df.groupby('region').agg({'asst_id':'count'}).reset_index()
gb

,region,asst_id
0,Hogwarts,69953
1,Middle Earth,24146
2,Milky Way,5898
3,Missing,3


In [13]:
gb = df.groupby('country').agg({'asst_id':'count'}).reset_index()
gb

,country,asst_id
0,Acrux,15
1,Alcor,5
2,Aldebaran,4
3,Altair,3
4,Antares,25
5,Arcturus,21
6,Becrux,30
7,Bellatrix,1
8,Betelgeuse,1
9,Bree,3


---
# 6. analysis 2
* Contact type, Repeat complaints and Outside contract
* The data is rolled up at Contact type and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Repeat complaints and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Outside contract and count of asset id is taken (the sum of count of asset id is 100,000).

In [14]:
gb = df.groupby('contact_type').agg({'asst_id':'count'}).reset_index()
gb

,contact_type,asst_id
0,CHAT,14425
1,EMAIL,1281
2,Unknown,118
3,VOICE,69757
4,Voice,14405


In [15]:
gb = df.groupby('repeat compaint').agg({'asst_id':'count'}).reset_index()
gb

,repeat compaint,asst_id
0,multiple,34742
1,single,65258


In [16]:
gb = df.groupby('outside contract').agg({'asst_id':'count'}).reset_index()
gb

,outside contract,asst_id
0,inside,83808
1,outside,16192


---
# 7. analysis 3
* Log and Issue 
* The data is rolled up at Issues and count of asset id is taken (the sum of count of asset id is 100,000). 

In [17]:
df['issue_type'] = df['issue_type'].str.replace(" ","")
gb = df.groupby('issue_type').agg({'asst_id':'count'}).reset_index()
gb

,issue_type,asst_id
0,Adobe,153
1,"Audio,Video,Speakers,TVTuner",8492
2,Backplane,217
3,Backup/Archive/DataManagement,21
4,Backup/RecoverySoftware,199
5,Battery/CMOS,2943
6,"Burn,Playback,Audio",108
7,"CFA,MFA(CustomerorManufacturer)FieldAction",88
8,Cancel&Rebook,4
9,ChangedMind,15


---
# 8. analysis 4
* Diagnostics, Repair type and Contact manager
* The data is rolled up at Diagnostics and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Repair type and count of asset id is taken (the sum of count of asset id is 100,000). The data is rolled up at Contact manager and count of asset id is taken (the sum of count of asset id is 100,000).

In [18]:
gb = df.groupby('diagnostics').agg({'asst_id':'count'}).reset_index()
gb

,diagnostics,asst_id
0,Effective_Usage,22812
1,InEffective_Usage,7238
2,Not_Considered,2485
3,Not_USED,67465


In [19]:
gb = df.groupby('repair_type').agg({'asst_id':'count'}).reset_index()
gb

,repair_type,asst_id
0,Hard,59495
1,Soft,40505


In [20]:
gb = df.groupby('contact_manager_flg').agg({'asst_id':'count'}).reset_index()
gb

,contact_manager_flg,asst_id
0,0,98757
1,1,1243


---
# 9. analysis 5
* Only those rows are kept where the contact_manager_flg is 1
* Diagnostics (Effective_Usage and InEffective_Usage) is preformed whenever the contact manager flag is 1 

In [21]:
dff = df[df['contact_manager_flg']==1]
gb = dff.groupby('diagnostics').agg({'asst_id':'count'}).reset_index()
gb

,diagnostics,asst_id
0,Effective_Usage,1171
1,InEffective_Usage,72
